In [4]:
import torch
import pygame
import numpy as np
import os

# Directory for saving frames

frame_directory = "frames"
os.makedirs(frame_directory, exist_ok=True)

# Ensure that CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Simulation parameters
width, height = 400, 400
Nx, Ny = 100, 100
dx, dy = 1.0 / Nx, 1.0 / Ny
dt = 0.00001  # Time step
viscosity = 0.2

# Initialize fields on GPU
u = torch.zeros((Ny, Nx), device=device, dtype=torch.float32)
v = torch.zeros((Ny, Nx), device=device, dtype=torch.float32)
p = torch.zeros((Ny, Nx), device=device, dtype=torch.float32)
rho = 2*(torch.ones((Ny, Nx), device=device, dtype=torch.float32))

def update_velocity(u, v, p, rho, dt, dx, dy, viscosity):
    un = u.clone()
    vn = v.clone()

    # Central differences in space, forward difference in time
    # Update u-velocity
    u[1:-1, 1:-1] = (un[1:-1, 1:-1] -
                     un[1:-1, 1:-1] * dt / dx * (un[1:-1, 1:-1] - un[1:-1, :-2]) -
                     vn[1:-1, 1:-1] * dt / dy * (un[1:-1, 1:-1] - un[:-2, 1:-1]) -
                     dt / (2 * rho[1:-1, 1:-1] * dx) * (p[1:-1, 2:] - p[1:-1, :-2]) +
                     viscosity * (dt / dx**2 * (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, :-2]) +
                                  dt / dy**2 * (un[2:, 1:-1] - 2 * un[1:-1, 1:-1] + un[:-2, 1:-1])))

    # Update v-velocity
    v[1:-1, 1:-1] = (vn[1:-1, 1:-1] -
                     un[1:-1, 1:-1] * dt / dx * (vn[1:-1, 1:-1] - vn[1:-1, :-2]) -
                     vn[1:-1, 1:-1] * dt / dy * (vn[1:-1, 1:-1] - vn[:-2, 1:-1]) -
                     dt / (2 * rho[1:-1, 1:-1] * dy) * (p[2:, 1:-1] - p[:-2, 1:-1]) +
                     viscosity * (dt / dx**2 * (vn[1:-1, 2:] - 2 * vn[1:-1, 1:-1] + vn[1:-1, :-2]) +
                                  dt / dy**2 * (vn[2:, 1:-1] - 2 * vn[1:-1, 1:-1] + vn[:-2, 1:-1])))

    return u, v

def pressure_poisson(p, u, v, rho, dx, dy):
    pn = p.clone()
    for q in range(10):
        pn = p.clone()

        p[1:-1, 1:-1] = (((pn[1:-1, 2:] + pn[1:-1, :-2]) * dy**2 +
                          (pn[2:, 1:-1] + pn[:-2, 1:-1]) * dx**2) /
                          (2 * (dx**2 + dy**2)) -
                          rho[1:-1, 1:-1] * dx**2 * dy**2 / (2 * (dx**2 + dy**2)) *
                          (((u[1:-1, 2:] - u[1:-1, :-2]) / (2 * dx) + 
                            (v[2:, 1:-1] - v[:-2, 1:-1]) / (2 * dy)) ** 2))

        # Boundary conditions for pressure
        p[:, -1] = p[:, -2]
        p[0, :] = p[1, :]
        p[:, 0] = p[:, 1]
        p[-1, :] = 0

    return p

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

running = True
frame_number = 0

while running:
    # Velocity boundary condition
    u[:, 0] = 1
    v[:, 0] = 1
    # Update velocity and pressure
    for i in range(50):
        u, v = update_velocity(u, v, p, rho, dt, dx, dy, viscosity)
        p = pressure_poisson(p, u, v, rho, dx, dy)

    # Transfer data to CPU for visualization and convert to NumPy
    u_cpu = u.cpu().numpy()
    v_cpu = v.cpu().numpy()

    # Visualization
    for i in range(Nx):
        for j in range(Ny):
            color = min(255, max(0, int(255 * np.sqrt(u_cpu[j, i]**2 + v_cpu[j, i]**2))))
            pygame.draw.rect(screen, (color, color, color), (i * width / Nx, j * height / Ny, width / Nx, height / Ny))

    pygame.display.flip()
    clock.tick(60)
    
    # Save frame
    pygame.image.save(screen, os.path.join(frame_directory, f"frame_{frame_number:04d}.png"))
    frame_number += 1

    # Check for Pygame quit events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()
